In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from bugFixLogic import bugFix

In [2]:
# import bugFixLogic_indexer as bugFix_indexer

In [ ]:
# bugFix_indexer.bugFix()

In [3]:
bugFix()

[[[]], [[1, 2, 6, 72, 7, 33, 4]], [[3, 1, 4, 1, 5, 9, 2, 6, 5, 3, 5, 8, 9, 7, 9, 3]], [[5, 4, 3, 2, 1]], [[5, 4, 3, 1, 2]]]
[[[]], [[1, 2, 4, 6, 7, 33, 72]], [[1, 1, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, 9, 9]], [[1, 2, 3, 4, 5]], [[1, 2, 3, 4, 5]]]
The folder 'FauxPyReport_SearchBasedBugFixing_sbfl_statement_2024_05_23_21_23_25_284657' has been deleted.
File '10.txt' copied from 'testcases/BuggyPrograms' to 'testcases/GeneratedTests'.
File 'test.py' created in 'testcases/GeneratedTests' with the converted Python code.
returned from fault localization



KeyboardInterrupt: 

In [5]:
# read faiss index and query

import numpy as np
import faiss

# Load the index
index = faiss.read_index('../hnsw_index.index')

# Assume we have a numpy array `xq` for querying
# xq = np.array([[1, 1, 1], [2, 2, 2], [3, 3, 3]], dtype=np.float32)

# Perform a search
# D, I = index.search(xq, k=5)  # search the top 5 similar vectors

# print(I)  # indices of the most similar vectors
# print(D)  # distances to the most similar vectors

: 

In [ ]:
import torch
from unixcoder import UniXcoder
import numpy as np
DEVICE = torch.device("cpu")
model = UniXcoder("microsoft/unixcoder-base")
model.to(DEVICE)

#returns numpy array of embeddings
def get_embeddings(model,tokens,device=DEVICE):
  tokens_ids = model.tokenize([tokens],max_length=512,mode="<encoder-only>")
  source_ids = torch.tensor(tokens_ids).to(device)
  _,embeddings = model(source_ids)
  return embeddings.detach()

In [19]:
from torch.utils.data import DataLoader

def batch_tokenize(token_list, max_length=512, mode="<encoder-only>"):
    # Tokenize multiple tokens in batches
    token_ids = model.tokenize(token_list, max_length=max_length, mode=mode)
    return torch.tensor(token_ids)

def fitness_bug_code(tokens_list, device=DEVICE, batch_size=64):
    dataloader = DataLoader(tokens_list, batch_size=batch_size)
    distances = []
    for batch_tokens in dataloader:
        token_ids = batch_tokenize(batch_tokens).to(device)
        _, embeddings = model(token_ids)
        distances.append(embeddings.detach().cpu())
    distances = torch.cat(distances)
    D, I = index.search(distances, k=1)
    return D

In [18]:
emb = get_embeddings(model,"print(y)")

In [4]:
D, I = index.search(emb, k=5)  # search the top 5 similar vectors

print(I)  # indices of the most similar vectors
print(D)  # distances to the most similar vectors

NameError: name 'index' is not defined